# Assignment Problem

In [370]:
import pandas as pd   
import warnings; warnings.simplefilter('ignore')
import numpy as np

In [371]:
df=pd.read_csv("assign1.csv")
df

,0,1,2,3
0,8,7,9,9
1,5,2,7,8
2,6,1,4,9
3,2,3,2,6


In [372]:

#df.append(pd.Series(dict(df.min()),name='min_val'))



In [373]:
def get_balanced(df):
    jobs=df.columns
    operators=df.index  
    jobs_len=len(jobs)
    operators_len=len(operators)
    if jobs_len==operators_len:
        return df
    elif jobs_len<operators_len:
        df.insert(jobs_len, str(jobs_len), [0 for i in range(operators_len)])
        return df
    else:
        return df.append({job: 0 for job in jobs}, ignore_index=True)
        
        
        
    

In [374]:
def row_reduction(df,jobs):         
    df['min_val']=df.min(axis=1)
    for job in jobs:
        df[job]-=df['min_val']
    return df


def col_reduction(df,operators):    
    df=df.append(pd.Series(dict(df.min()),name='min_val'))  
    df['min_val']['min_val']=0
    for operator in operators:
        df.loc[operator]-= df.loc['min_val']
    return df

In [375]:
def GetKey(dic,val):
    for key, value in dic.items():
        if val == value:
            return key

In [376]:
def row_scanning(df,operators,allocated_zeroes):  
    df.loc['is_striked']='-'
    df.loc['is_striked','min_val']='o'
    df['selected_0']=None
    allocated_zeroes=0
    for i in operators:
        for operator in operators:
            if list(df.loc[operator,df.loc['is_striked']=='-']).count(0)==1:
                job=GetKey(dict(df.loc[operator,df.loc['is_striked']=='-']),0)
                df[job]['is_striked']='x'
                df['selected_0'][operator]=job
                allocated_zeroes+=1
    return df, allocated_zeroes
            
  

    
        
def col_scanning(df,jobs, allocated_zeroes):
    df['is_striked']='-'   
    df['is_striked']['min_val']='o'
    df.loc['selected_0']=None  
    for i in jobs:
        for job in jobs:
            if df[job]['is_striked']=='-':            
                if list(df.loc[df['is_striked']=='-',job]).count(0)==1:                
                    operator=GetKey(dict(df.loc[df['is_striked']=='-',job]),0)
                    df['is_striked'][operator]='x'
                    df[job]['selected_0']=operator
                    allocated_zeroes+=1
    return df, allocated_zeroes

#nrows=no. of rows= len(operators)
def is_optimal(nrows, allocated_zeroes):
    return nrows==allocated_zeroes   
    

In [377]:
def modify_table(df,jobs,operators):
    min_val=999999999
    for job in jobs:
        for operator in operators:
            if df['is_striked'][operator]=='-' and df[job]['is_striked']=='-':
                if min_val>df[job][operator]:
                    min_val=df[job][operator]
    if min_val==999999999:
        raise Exception("min uncovered value is infinity")    
   
    print("Minimum value of uncovered element is", min_val)
    for job in jobs:
        for operator in operators:
            if df['is_striked'][operator]=='-' and df[job]['is_striked']=='-':                
                df[job][operator]-=min_val
        
        
#     df2a=df.loc[:,df['is_striked']=='x']
#     df2=df2a.loc[df['is_striked']=='x',:]
    print("--------------------")
    print("Intersection elements:")
    for job in jobs:
        for operator in operators:
            if df['is_striked'][operator]=='x' and df[job]['is_striked']=='x':                
                df[job][operator]+=min_val
                print("row_id:",operator,"col_id:",operator,"value:",df[job][operator])
#     print(df2)
#     for job in df2.columns:
#         for operator in df2.index:
#             df[job][operator]+=min_val
    print("--------------------")
    return df
    
    

In [378]:
def optimal_assignment(initial_df, df, jobs,operators):
    min_cost=0
    for operator in operators:
        if df['selected_0'][operator] is not None:
            job=df['selected_0'][operator]
            print("Job:",job,"operator:",operator,"minimum cost:",initial_df[job][operator])
            min_cost+=initial_df[job][operator]
            
    for job in jobs:
        if df[job]['selected_0'] is not None:
            operator=df[job]['selected_0']
            print("Job:",job,"operator:",operator,"minimum cost:",initial_df[job][operator])
            min_cost+=initial_df[job][operator]
            
    print("Total minimum cost is:",min_cost)
    
    

In [379]:
def assignment_problem(df):
    df=get_balanced(df)
    operators=list(df.index)
    jobs=list(df.columns)
    initial_df=df.copy(deep=True)
    print("assignment problem after balancing:")
    print(df)
    print("___________________________________________________")
    print("table after row reduction:")
    df=row_reduction(df,jobs)
    print(df)
    print("___________________________________________________")
    print("table after column reduction:")
    df=col_reduction(df,operators)
    print(df)
#     df.loc['is_striked']='-'
#     df.loc['min_val','is_striked']='o'
#     df['is_striked']='-'   
#     df['is_striked']['min_val']='o'
    allocated_zeroes=0
    for i in range(10):
        for j in range(2):
            print("___________________________________________________")
            print("table after row scanning:")
            df,allocated_zeroes=row_scanning(df,operators,allocated_zeroes)
            print(df)
            print("___________________________________________________")
            print("table after col scanning:")
            df,allocated_zeroes=col_scanning(df,jobs, allocated_zeroes)
            print(df)
        if is_optimal(len(operators), allocated_zeroes):
            print("___________________________________________________")
            print("Optimality reached")
            optimal_assignment(initial_df, df, jobs,operators)
            break
        else:
            print("___________________________________________________")
            df=modify_table(df,jobs,operators)
            print("Table after modication:")
            print(df)
            
        
            
        
        

In [355]:
df=pd.read_csv("assign1.csv")
assignment_problem(df)

assignment problem after balancing:
   0  1  2  3
0  8  7  9  9
1  5  2  7  8
2  6  1  4  9
3  2  3  2  6
___________________________________________________
table after row reduction:
   0  1  2  3  min_val
0  1  0  2  2        7
1  3  0  5  6        2
2  5  0  3  8        1
3  0  1  0  4        2
___________________________________________________
table after column reduction:
         0  1  2  3  min_val
0        1  0  2  0        7
1        3  0  5  4        2
2        5  0  3  6        1
3        0  1  0  2        2
min_val  0  0  0  2        0
___________________________________________________
Minimum value of uncovered element is 1
--------------------
Intersection elements:
row_id: 3 col_id: 3 value: 2
row_id: 3 col_id: 3 value: 3
--------------------
Table after modication:
            0     1     2     3 min_val is_striked selected_0
0           0     0     1     0       7          -          3
1           2     0     4     4       2          -          1
2           4     0

In [356]:
df=pd.read_csv("assign2.csv")
assignment_problem(df)

assignment problem after balancing:
    0   1   2   3   4
0  85  93  24   6  10
1  70  57  20   1  19
2  65  37  25  89  77
3  68  97  23  84  38
4   0   0   0   0   0
___________________________________________________
table after row reduction:
    0   1   2   3   4  min_val
0  79  87  18   0   4        6
1  69  56  19   0  18        1
2  40  12   0  64  52       25
3  45  74   0  61  15       23
4   0   0   0   0   0        0
___________________________________________________
table after column reduction:
          0   1   2   3   4  min_val
0        79  87  18   0   4        6
1        69  56  19   0  18        1
2        40  12   0  64  52       25
3        45  74   0  61  15       23
4         0   0   0   0   0        0
min_val   0   0   0   0   0        0
___________________________________________________
Minimum value of uncovered element is 4
--------------------
Intersection elements:
row_id: 4 col_id: 4 value: 4
row_id: 4 col_id: 4 value: 4
--------------------
Table after

In [380]:
df=pd.read_csv("assign3.csv")
assignment_problem(df)

assignment problem after balancing:
    0   1   2   3  4
0  57  97  56  43  0
1  33  59  63  97  0
2  22  22  33  11  0
3  31  45  84  71  0
4  98   7  90  96  0
___________________________________________________
table after row reduction:
    0   1   2   3  4  min_val
0  57  97  56  43  0        0
1  33  59  63  97  0        0
2  22  22  33  11  0        0
3  31  45  84  71  0        0
4  98   7  90  96  0        0
___________________________________________________
table after column reduction:
          0   1   2   3  4  min_val
0        35  90  23  32  0        0
1        11  52  30  86  0        0
2         0  15   0   0  0        0
3         9  38  51  60  0        0
4        76   0  57  85  0        0
min_val  22   7  33  11  0        0
___________________________________________________
table after row scanning:
             0   1   2   3  4 min_val selected_0
0           35  90  23  32  0       0          4
1           11  52  30  86  0       0       None
2            0  15  

In [358]:
df=pd.read_csv("assign4.csv")
assignment_problem(df)

assignment problem after balancing:
    0   1   2   3   4
0  15  18   1  40  93
1  86  86  85  86  86
2  19  19  79  54  78
3  56  80  35  27  14
4  54  53  47  99  42
___________________________________________________
table after row reduction:
    0   1   2   3   4  min_val
0  14  17   0  39  92        1
1   1   1   0   1   1       85
2   0   0  60  35  59       19
3  42  66  21  13   0       14
4  12  11   5  57   0       42
___________________________________________________
table after column reduction:
          0   1   2   3   4  min_val
0        14  17   0  38  92        1
1         1   1   0   0   1       85
2         0   0  60  34  59       19
3        42  66  21  12   0       14
4        12  11   5  56   0       42
min_val   0   0   0   1   0        0
___________________________________________________
Minimum value of uncovered element is 1
--------------------
Intersection elements:
row_id: 2 col_id: 2 value: 61
row_id: 2 col_id: 2 value: 35
row_id: 2 col_id: 2 value: 60
